# Resolução de sistemas de equações lineares

## Importações

In [1]:
import numpy as np

## Funções Auxiliares

In [2]:
def pivoting_function(A, B, base, length):
  # choose pivot
  max = (abs(A[base, base]), base)
  for row in range(base, length - 1):
    if abs(A[row + 1, 0]) > max[0]:
      max = (A[row + 1, 0], row + 1)

  if max[0] != base:
    aux = np.copy(A[base])
    A[base] = A[max[1]]
    A[max[1]] = aux

    aux = np.copy(B[base])
    B[base] = B[max[1]]
    B[max[1]] = aux

## Definições

In [3]:
# Ex1

A1 = np.array([
          [0, 1, 3, 2, 4],
          [8, 2, 9, -1, 2],
          [5, 1, 1, 7, 2],
          [-2, 4, 5, 1, 0],
          [7, -3, 2, -4, 1]
          ], dtype='float')  

B1 = np.array([3, -5, 6, -1, 8], dtype='float')

# Ex2

A2 = np.array([
          [2, 2, 1],
          [3, 3, 2],
          [3, 2, 1]
          ], dtype='float')  
          
A3 = np.array([
          [3, 2, 1],
          [2, 2, 1],
          [3, 3, 1]
          ], dtype='float')  

A4 = np.array([
          [2, 1, 3],
          [4, 3, 8],
          [6, 7, 17]
          ], dtype='float')  

# Ex3

A5 = np.array([
          [1, 2, 0, -3, 3],
          [2, 5, -1, 1, 4],
          [-3, -1, 50, 1, -19],
          [0, 1, 1, 6, 0],
          [3, 4, -19, 0, 39]
          ], dtype='float')  

B5 = np.array([17, 41, -45, 30, 51], dtype='float')

## Algoritmos

### Eliminação de Gauss s/ Pivotamento

In [14]:
def gauss_elimination(coeficients, constants, precision, pivoting=False, factors=False):
  A = np.copy(coeficients)
  B = np.copy(constants)
  length = len(A) - 1

  factors_list = np.identity(len(A), dtype='float')

  # superior matrix
  for base in range(length):
    if pivoting:
      pivoting_function(A, B, base, length)
      
    for row in range(base + 1, len(A)):
      # pivot validation
      if A[base, base] == 0 and not pivoting:
        print('Não foi possível resolver o sistema, pivô nulo!')
        print(f'linha: {base}, coluna: {base}')
        return None

      factor = A[row, base] / A[base, base]
      factors_list[row, base] = factor
      
      A[row, base] = 0

      for col in range(base + 1, len(A)):
        A[row, col] = A[row, col] - factor * A[base, col]

      B[row] = B[row] - (factor * B[base])

  # resolution
  x = np.zeros(len(A), dtype='float')
  x[length] = B[length] / A[length, length]

  for row in range(length, 0, -1):
    s = 0
    for col in range(row + 1, len(A)):
      s = s + A[row, col] * x[col]
      x[row] = (B[row] - s) / A[row, row]

  #np.set_printoptions(precision=precision)

  if factors:
    return A, B, x, factors_list
  else:
    return A, B, x


In [5]:
def lu_decomposition(coeficients):
  A = np.copy(coeficients)
  length = len(A) - 1

  gauss = gauss_elimination(A, np.zeros(len(A), dtype='float'), 0, factors=True)
  lower = gauss[0]
  upper = gauss[3]

  return lower, upper

## Exercícios

### Exercício 1 

Resolva o SELA pelo método de Eliminação de Gauss sem pivotamento e com pivotamento parcial, retendo, durante as eliminações, cinco algarismos depois da vírgula. Compare os resultados. (A1, B1)

In [10]:
gauss_elimination(A1, B1, 5, pivoting=False)

Não foi possível resolver o sistema, pivô nulo!
linha: 0, coluna: 0


In [16]:
gauss_elimination(A1, B1, 5, pivoting=True)

(array([[  8.        ,   2.        ,   9.        ,  -1.        ,
           2.        ],
        [  0.        ,   1.        ,   3.        ,   2.        ,
           4.        ],
        [  0.        ,   0.        ,  -3.875     ,   8.125     ,
           1.75      ],
        [  0.        ,   0.        ,   0.        , -21.35483871,
         -20.32258065],
        [  0.        ,   0.        ,   0.        ,   0.        ,
          -0.74622356]]),
 array([ -5.        ,   3.        ,   9.875     , -31.67741935,
         12.46223565]),
 array([  0.        , -43.98380567,  26.34412955,  17.37651822,
        -16.70040486]))

Podemos verificar que, com o sistema linear fornecido pelo exercício, podemos apenas resolver utilizando a eliminação de Gauss com pivotamento pois se enquadra exatamente em um das motivações para se utilizar o pivotamento, que seria o pivô escolhido, nesse caso o primeiro elemento da primeira coluna, acabar resultando em uma divisão por zero, o que inviabiliza a comparação das duas variações

### Exercício 2

Quais das matrizes (A2, A3, A4) podem ser decompostas na forma LU ? Decompor as que forem possíveis.